<a href="https://colab.research.google.com/github/aleks-haksly/Postypashka/blob/main/DE/01%20HW/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [4]:
spark

#**RDD**

# Вспомогательные функции


In [5]:
!wget "https://raw.githubusercontent.com/aleks-haksly/Postypashka/main/DE/01%20HW/datasets/RDD/dataset/order/order.csv" -q
!wget "https://raw.githubusercontent.com/aleks-haksly/Postypashka/main/DE/01%20HW/datasets/RDD/dataset/customer/customer.csv" -q
!wget "https://raw.githubusercontent.com/aleks-haksly/Postypashka/main/DE/01%20HW/datasets/RDD/dataset/product/product.csv" -q

In [6]:
path_to_order = "/content/order.csv"
path_to_customer = "/content/customer.csv"
path_to_product = "/content/product.csv"

def createRDD(spark:SparkSession, path):
  return spark.sparkContext.textFile(path)\
                    .map(lambda x: x.split('\t'))

  # Задача 1
  * 1. Загрузить в RDD файл src/test/resources/input/order
  * 2. Выбрать строки в которых поле статус заказа delivered
  * 3. Выбрать ключ (customerID), значение (numberOfProduct, 1)
  * 4. Написать функцию reduce для расчёта
  * 5. Применит к RDD из п.3 метод reduceByKey c функцией из п.4
  * 6. Вывести или записать результат

In [7]:
orders = spark.sparkContext.textFile(path_to_order)
orders = orders.map(lambda x: x.split('\t'))\
              .filter(lambda x: 'delivered' in x[5])\
              .map(lambda x: (x[0], (x[3],1)))\
              .reduceByKey(lambda x,y: ((int(x[0])+int(y[0])),(x[1]+y[1]))).take(5)

print(orders)

[('1', (3000, 9)), ('4', (1300, 8)), ('2', (1250, 11)), ('3', ('100', 1)), ('5', (1300, 8))]


# Задача 2
   * 1. Загрузить в RDD файл src/test/resources/input/order
   * 2. Распарсить строки RDD
   * 3. Выбрать ключ поле (customerID), в значение (orderDate, numberOfProduct, productID)
   * 4. Загрузить в RDD файл customer
   * 5. Распарсить строки RDD
   * 6. Выбрать ключ поле (id), в значение (name)
   * 7. Выполнить внутреннее соединение RDD из п.6 и п.9
   * 8. Выбрать ключ (productID), в значение (customer.name, orderDate,  numberOfProduct)
   * 9. Загрузить в RDD файл product.csv
   * 10. Распарсить строки RDD
   * 11. Выбрать ключ (id), значение (price)
   * 12. Выполнить внутреннее соединение с RDD из п.11 и п.14
   * 13. Выбрать ключ (customer.name, order.orderDate), значение (order.numberOfProduct * product.price)
   * 14. Расчитать сумму в значении по ключу
   * 15. Вывести результат

In [8]:
pd.read_csv(path_to_order, sep='\t', header=None, names=['customerID_0', 'oderID_1', 'productID_2', 'numberOfProduct_3', 'orderDate_4', 'status_5']).head()

,customerID_0,oderID_1,productID_2,numberOfProduct_3,orderDate_4,status_5
0,1,21,3,500,2018-02-23,delivered
1,1,22,1,300,2018-02-23,delivered
2,1,23,2,300,2018-02-23,delivered
3,1,24,1,500,2018-03-23,delivered
4,1,25,2,300,2018-03-23,delivered


In [9]:
orders = spark.sparkContext.textFile(path_to_order)
orders = orders.map(lambda x: x.split('\t'))\
              .map(lambda x: (x[0], (x[4], x[3], x[2])))\


print(orders.take(5))

[('1', ('2018-02-23', '500', '3')), ('1', ('2018-02-23', '300', '1')), ('1', ('2018-02-23', '300', '2')), ('1', ('2018-03-23', '500', '1')), ('1', ('2018-03-23', '300', '2'))]


In [10]:
customer = spark.sparkContext.textFile(path_to_customer)
customer = customer.map(lambda x: x.split('\t'))\
              .map(lambda x: (x[0], (x[1])))\


print(customer.take(5))

[('1', 'John'), ('2', 'Philip'), ('3', 'Vasili'), ('4', 'Anastasia'), ('5', 'Robert')]


In [11]:
joined_rdd = orders.join(customer)
print(joined_rdd.take(5))

[('4', (('2018-05-01', '200', '1'), 'Anastasia')), ('4', (('2018-05-01', '200', '2'), 'Anastasia')), ('4', (('2018-05-01', '200', '3'), 'Anastasia')), ('4', (('2018-06-01', '200', '1'), 'Anastasia')), ('4', (('2018-06-01', '200', '2'), 'Anastasia'))]


In [12]:
joined_rdd = orders.join(customer)
print(joined_rdd.take(5))

[('4', (('2018-05-01', '200', '1'), 'Anastasia')), ('4', (('2018-05-01', '200', '2'), 'Anastasia')), ('4', (('2018-05-01', '200', '3'), 'Anastasia')), ('4', (('2018-06-01', '200', '1'), 'Anastasia')), ('4', (('2018-06-01', '200', '2'), 'Anastasia'))]


In [13]:
joined_rdd = joined_rdd.map(lambda x: (x[1][0][2], (x[1][1], x[1][0][0], x[1][0][1])))
print(joined_rdd.take(5))

[('1', ('Anastasia', '2018-05-01', '200')), ('2', ('Anastasia', '2018-05-01', '200')), ('3', ('Anastasia', '2018-05-01', '200')), ('1', ('Anastasia', '2018-06-01', '200')), ('2', ('Anastasia', '2018-06-01', '200'))]


In [14]:
product = spark.sparkContext.textFile(path_to_product)
product = product.map(lambda x: x.split('\t'))\
              .map(lambda x: (x[0], (x[2])))
print(product.take(5))

[('1', '45990'), ('2', '49990'), ('3', '75990'), ('4', '29990'), ('5', '54990')]


In [15]:
joined_rdd = joined_rdd.join(product)
print(joined_rdd.take(5))

[('4', (('Sara', '2017-07-21', '100'), '29990')), ('4', (('Philip', '2017-10-16', '50'), '29990')), ('4', (('Robert', '2017-11-01', '200'), '29990')), ('4', (('Robert', '2017-12-01', '200'), '29990')), ('4', (('Robert', '2018-01-01', '200'), '29990'))]


In [16]:
joined_rdd_1 = joined_rdd.map(lambda x: ((x[1][0][0], x[1][0][1]), int(x[1][0][2]) * int(x[1][1])))
print(joined_rdd_1.take(5))

[(('Sara', '2017-07-21'), 2999000), (('Philip', '2017-10-16'), 1499500), (('Robert', '2017-11-01'), 5998000), (('Robert', '2017-12-01'), 5998000), (('Robert', '2018-01-01'), 5998000)]


In [17]:
joined_rdd_1.reduceByKey(lambda x,y: x+y).collect()

[(('Sara', '2017-07-21'), 64880000),
 (('Robert', '2017-12-01'), 5998000),
 (('Philip', '2018-04-16'), 19594000),
 (('Robert', '2017-11-01'), 16996000),
 (('Robert', '2018-02-01'), 2999000),
 (('Anastasia', '2018-08-01'), 15198000),
 (('Vasili', '2018-06-01'), 26795000),
 (('Anastasia', '2018-06-01'), 19196000),
 (('Robert', '2017-12-02'), 10998000),
 (('Anastasia', '2018-07-01'), 9598000),
 (('Robert', '2018-02-02'), 5499000),
 (('Philip', '2017-10-16'), 41543500),
 (('Robert', '2018-01-01'), 11497000),
 (('Anastasia', '2018-05-01'), 34394000),
 (('Vasili', '2018-05-01'), 4599000),
 (('John', '2018-02-23'), 66789000),
 (('John', '2018-03-23'), 60789000),
 (('John', '2018-05-23'), 15497000),
 (('John', '2018-04-23'), 43592000)]

# Задача 3
 * 1. Загрузить в RDD файл src/test/resources/input/product
 * 2. Распарсить строки в RDD
 * 3. Выбрать ключ поле id, в значение name
 * 4. Загрузить в RDD файд src/test/resources/input/order
 * 5. Распарсить строки в RDD
 * 6. Выбрать ключ поле productID, в значение numberOfProduct
 * 7. Посчитать кол-во проданных продуктов
 * 8. Выполнить левое соединение двух RDD
 * 9. Выполнить фильтрацию и оставить только те строки где значение numberOfProducts 0 или None
 * 10. Вывести результат или записать в директорию

In [22]:
product = spark.sparkContext.textFile(path_to_product)
product = product.map(lambda x: x.split('\t'))\
              .map(lambda x: (x[0], (x[1])))
print(product.take(5))

[('1', 'Apple iPhone 7'), ('2', 'Apple iPhone 8'), ('3', 'Apple iPhone 10'), ('4', 'Apple iPad mini 4'), ('5', 'Apple iPad Pro')]


In [54]:
orders = spark.sparkContext.textFile(path_to_order)
orders = orders.map(lambda x: x.split('\t'))\
              .map(lambda x: (x[2], int(x[3])))\

print(orders.take(5))

[('3', 500), ('1', 300), ('2', 300), ('1', 500), ('2', 300)]


In [56]:
orders_grouped_by_product = orders.reduceByKey(lambda x, y: x + y)\
                                  .map(lambda x: (x[0], x[1]))
orders_grouped_by_product.take(5)

[('1', 2700), ('4', 850), ('8', 1250), ('100', 0), ('3', 2050)]

In [59]:
joined_rdd = product.leftOuterJoin(orders_grouped_by_product)
print(joined_rdd.take(5))

[('4', ('Apple iPad mini 4', 850)), ('10', ('Apple HomePod', None)), ('3', ('Apple iPhone 10', 2050)), ('6', ('Apple MacBook Air', 100)), ('7', ('Apple MakBook Pro', 100))]


In [58]:
joined_rdd.filter(lambda x: x[1][1] == None or x[1][1] == 0)\
          .collect()

[('10', ('Apple HomePod', None)), ('9', ('Apple EarPods', None))]

# Задача 4
 * 1. Загрузить в RDD файд src/test/resources/input/order.csv
 * 2. Распарсить строки в RDD
 * 3. Выбрать только те транзакции у которых статус delivered
 * 4. Выбрать ключ (customerID), значение (numberOfProducts)
 * 5. Выполнить группировку по ключу
 * 6. Посчитать сумму по значению и разделить на размер коллекции
 * 7. Вывести результат или записать в директорию

# **DATAFRAME**

# Вспомогательные функции

In [19]:
from pyspark.sql.types import *
class Parameters:
  path_laptop = "/content/drive/MyDrive/DE/LabDF/dataset/laptop/laptop.txt"
  path_pc = "/content/drive/MyDrive/DE/LabDF/dataset/pc/pc.txt"
  path_printer = "/content/drive/MyDrive/DE/LabDF/dataset/printer/printer.txt"
  path_product = "/content/drive/MyDrive/DE/LabDF/dataset/product/product.txt"

  table_laptop = "laptop"
  table_pc = "pc"
  table_printer = "printer"
  table_product = "product"

  def createTable(self, name:str, spark:SparkSession, structType: StructType, path:str, delimiter: str = ','):
      spark.read\
            .options(delimiter = delimiter, nullValue = "\\N")\
            .schema(structType)\
            .csv(path)\
            .createOrReplaceTempView(name)
  def initTables(self, spark:SparkSession):
    self.createTable(Parameters.table_pc, spark, PC.structType, Parameters.path_pc)
    self.createTable(Parameters.table_product, spark, Product.structType, Parameters.path_product)

In [20]:
class PC:
  structType = StructType([
      StructField("ID", IntegerType()),
      StructField("MODEL", IntegerType()),
      StructField("SPEED", IntegerType()),
      StructField("RAM", IntegerType()),
      StructField("C2", IntegerType()),
      StructField("C3", StringType()),
      StructField("PRICE", IntegerType()),
  ])
class Product:
  structType = StructType([
      StructField("Maker", StringType()),
      StructField("Model", StringType()),
      StructField("Type", StringType())
  ])

In [21]:
p = Parameters()
p.initTables(spark)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/drive/MyDrive/DE/LabDF/dataset/pc/pc.txt.

# Задача 1
Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью.
Вывести: speed, средняя цена.

SELECT pc.speed, AVG(pc.price)
FROM pc
WHERE pc.speed > 600
GROUP BY pc.speed
)

In [ ]:
from pyspark.sql.functions import col, avg
pc = spark.read.table("pc")\
              .select("speed","price")\
              .filter(col("price") > 600)\
              .groupBy("speed")\
              .agg(avg(col("price")).alias("avg_price"))\
              .select("speed","avg_price").show()

# Задача 2
Вывести все строки из таблицы Product, кроме трех строк с наименьшими номерами моделей и трех строк с наибольшими номерами моделей.

  Select maker, model, type from
  (
  Select
  row_number() over (order by model) p1,
  row_number() over (order by model DESC) p2,
  from Product
  ) t1
  where p1 > 3 and p2 > 3


In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc

product = spark.table("product")
MaxVal = Window.orderBy("model")
product1 = product.withColumn("p1", row_number().over(MaxVal))
MinVal = Window.orderBy(desc("model"))
product1.withColumn("p2", row_number().over(MinVal))\
        .select("maker", "model", "type")\
        .filter((col("p1")>3) & (col("p2")>3)).show()

# Задача 3
  Найти тех производителей ПК, все модели ПК которых имеются в таблице PC.

  SELECT p.maker
  FROM product p
  LEFT JOIN pc ON pc.model = p.model
  WHERE p.type = 'PC'
  GROUP BY p.maker
  HAVING COUNT(p.model) = COUNT(pc.model)
  

# Задача 4
Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

  SELECT DISTINCT maker
  FROM product
   WHERE model IN (
  SELECT model
  FROM pc
  WHERE ram = (
    SELECT MIN(ram)
    FROM pc
    )
  AND speed = (
    SELECT MAX(speed)
    FROM pc
    WHERE ram = (
     SELECT MIN(ram)
     FROM pc
     )
    )
  )
  AND
  maker IN (
  SELECT maker
  FROM product
  WHERE type='printer'
  )